Server communication game and game_loop.

In [91]:
## ignore this code, just used for submission

#Update your token
STUDENT_TOKEN = 'EUGENIOMARCHIORI'

import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'

  def get_board(self):
    return json.loads(self.state)

  def actions(self):
    return []

  def print_game(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
    while id == None:
        print('\033[92mCreating new game...\033[0m')
        id = new_game(game_type, multi_player)

    print('\033[92mJoining game with id: %s\033[0m' % id)
    player = join_game(game_type, id)

    print('\033[92mPlaying as %s\033[0m' % player)

    game = game_state(game_type, id, GameClass)
    print('\033[91mWaiting for the other player to join...\033[0m')
    while game.is_waiting():
        time.sleep(10)
        game = game_state(game_type, id, GameClass)

    while True:
        game = game_state(game_type, id, GameClass)
        if game is None:
            print("Failed to fetch the game state.")
            break  # Exit the loop if the game state couldn't be fetched

        game.print_game()
        if game.is_end():
            if game.player == '-':
                print('\033[94mdraw\033[0m')
            else:
                print('\033[92mwinner\033[0m' if game.player == player else '\033[91mloser\033[0m')
            return
        if game.player == player:
            print('Making next move...')
            move = solver(game)
            update_result = update_game(game_type, id, player, json.dumps(move))
            print(update_result)
        else:
            time.sleep(2)


MiniMax Implementation

In [92]:
def evaluate_board(board, player):
    score = 0
    opponent = 'o' if player == 'x' else 'x'
    for row in board:
        for cell in row:
            if cell.lower() == player.lower():
                score += 1
            elif cell.lower() == opponent.lower():
                score -= 1
    return score

def minimax(game, depth, player):
    if depth == 0 or game.is_end():
        return evaluate_board(json.loads(game.state), player), None

    if game.player == player:
        max_eval = float('-inf')
        best_move = None
        for action in game.actions():
            new_game = deepcopy(game)
            new_board = apply_move(json.loads(new_game.state), action, game.player)
            new_game.state = json.dumps(new_board)
            eval, _ = minimax(new_game, depth - 1, player)
            if eval > max_eval:
                max_eval = eval
                best_move = action
        return max_eval, best_move
    else:
        min_eval = float('inf')
        best_move = None
        for action in game.actions():
            new_game = deepcopy(game)
            new_board = apply_move(json.loads(new_game.state), action, game.player)
            new_game.state = json.dumps(new_board)
            eval, _ = minimax(new_game, depth - 1, player)
            if eval < min_eval:
                min_eval = eval
                best_move = action
        return min_eval, best_move

def apply_move(board, move, player):
    row_from, col_from, row_to, col_to = move
    new_board = deepcopy(board)
    new_board[row_to][col_to] = new_board[row_from][col_from]
    new_board[row_from][col_from] = '_'
    if abs(row_to - row_from) == 2:
        captured_row = row_from + (row_to - row_from) // 2
        captured_col = col_from + (col_to - col_from) // 2
        new_board[captured_row][captured_col] = '_'
    if (player.lower() == 'x' and row_to == 7) or (player.lower() == 'o' and row_to == 0):
        new_board[row_to][col_to] = player.upper()
    return new_board

def minimax_solver(game):
    _, best_move = minimax(game, 5, game.player)  # Assuming a depth of 3 for the Minimax search
    return best_move



Game specific code

In [93]:
import math
class MCTSNode:
    def __init__(self, state, parent=None, move=None):
        self.state = state
        self.parent = parent
        self.move = move
        self.children = []
        self.wins = 0
        self.visits = 0
        self.untried_actions = self.untried_moves()

    def untried_moves(self):
        return self.state.actions()

    def is_terminal_node(self):
        return self.state.is_end()

    def select_child(self):
        """ Select a child node with the highest UCB1 value """
        ucb1_values = [(child.wins / child.visits) + 2 * (2 * math.log(self.visits) / child.visits) ** 0.5 for child in self.children]
        return self.children[ucb1_values.index(max(ucb1_values))]

    def add_child(self, move, state):
        """ Add a new child node for this move """
        child_node = MCTSNode(state=state, parent=self, move=move)
        self.untried_actions.remove(move)
        self.children.append(child_node)
        return child_node

    def update(self, result):
        """ Update this node - one additional visit and result additional wins """
        self.visits += 1
        self.wins += result


def mcts_solver(game, iterations=1000):
    root = MCTSNode(state=game)

    for _ in range(iterations):
        node = root
        state = deepcopy(game)

        # Selection
        while node.untried_actions == [] and node.children != []:  # node is fully expanded and non-terminal
            node = node.select_child()
            state.play(node.move)

        # Expansion
        if node.untried_actions != []:
            move = random.choice(node.untried_actions) 
            state.play(move)
            node = node.add_child(move, state)  # add child and descend tree

        # Simulation
        while not state.is_end():
            possible_actions = state.actions()
            if possible_actions:
                state.play(random.choice(possible_actions))
            else:
                break  # No more actions possible, break out of the loop

        # Backpropagation
        while node is not None:
            if state.is_end():
                game_result = state.result(node.parent.state.player)
            else:
                game_result = 0.5  # default value or continue simulation
            node.update(game_result)
            node = node.parent

    # Return the move that was most visited
    return sorted(root.children, key=lambda c: c.visits)[-1].move



In [94]:
from functools import reduce
from copy import copy, deepcopy
import json
import random


class Checkers(Game):
  def __init__(self, state, status, player):
    super().__init__(state, status, player)

  def in_board(self, pos):
    return pos >= 0 and pos < 8

  def actions(self):
    board = self.get_board()
    player = self.player
    other_player = self.other_player()

    actions = []
    direction = 1 if player == 'X' else -1

    # Check for jumps
    for row in range(8):
        for col in range(8):
            if board[row][col].lower() == player.lower():
                # Check for possible jumps
                for d_row, d_col in [(2, 2), (2, -2), (-2, 2), (-2, -2)]:
                    new_row, new_col = row + d_row, col + d_col
                    if self.in_board(new_row) and self.in_board(new_col):
                        mid_row, mid_col = row + d_row // 2, col + d_col // 2
                        if board[mid_row][mid_col].lower() == other_player.lower() and board[new_row][new_col] == '_':
                            actions.append((row, col, new_row, new_col))

    if not actions:
        # Check for regular moves
        for row in range(8):
            for col in range(8):
                if board[row][col].lower() == player.lower():
                    # Check for possible moves
                    for d_row, d_col in [(1, 1), (1, -1), (-1, 1), (-1, -1)]:
                        new_row, new_col = row + d_row, col + d_col
                        if self.in_board(new_row) and self.in_board(new_col) and board[new_row][new_col] == '_':
                            actions.append((row, col, new_row, new_col))

    return actions


  def play(self, move):
    """Updates the game state with the given move"""
    board = self.get_board()
    row, col, new_row, new_col = move
    board[new_row][new_col] = board[row][col]
    board[row][col] = '_'

    # Remove captured piece in case of jump
    if abs(new_row - row) == 2:
        mid_row, mid_col = (row + new_row) // 2, (col + new_col) // 2
        board[mid_row][mid_col] = '_'

    self.state = json.dumps(board)
    self.player = self.other_player()

  def result(self, player):
    """Returns the game result from the perspective of the given player"""
    if self.is_end():
        if self.player == '-':
            return 0.5  # draw
        elif self.player == player:
            return 1  # win
        else:
            return 0  # loss
    return 0.5  # default value for non-terminal state or continue simulation

  def print_game(self):
    for row in self.get_board():
      print(row)

  def other_player(self):
    if self.player == 'X': return 'O'
    if self.player == 'O': return 'X'


  

In [103]:
game_loop(mcts_solver, Checkers, 'checkers', multi_player=False, id=None)


Creating new game...
Joining game with id: 3186
Playing as X
Waiting for the other player to join...
['_', 'x', '_', 'x', '_', 'x', '_', 'x']
['x', '_', 'x', '_', 'x', '_', 'x', '_']
['_', 'x', '_', 'x', '_', 'x', '_', 'x']
['_', '_', '_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_', '_', '_']
['o', '_', 'o', '_', 'o', '_', 'o', '_']
['_', 'o', '_', 'o', '_', 'o', '_', 'o']
['o', '_', 'o', '_', 'o', '_', 'o', '_']
Making next move...
b'Valid move'
['_', 'x', '_', 'x', '_', 'x', '_', 'x']
['x', '_', 'x', '_', 'x', '_', 'x', '_']
['_', '_', '_', 'x', '_', 'x', '_', 'x']
['x', '_', '_', '_', '_', '_', '_', '_']
['_', 'o', '_', '_', '_', '_', '_', '_']
['_', '_', 'o', '_', 'o', '_', 'o', '_']
['_', 'o', '_', 'o', '_', 'o', '_', 'o']
['o', '_', 'o', '_', 'o', '_', 'o', '_']
Making next move...
b'Valid move'
['_', 'x', '_', 'x', '_', 'x', '_', 'x']
['x', '_', 'x', '_', 'x', '_', 'x', '_']
['_', '_', '_', 'x', '_', '_', '_', 'x']
['x', '_', '_', '_', '_', '_', 'x', '_']
['_', 'o', 